<a href="https://colab.research.google.com/github/WonJunPark/spam_classification/blob/master/dacon_spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dacon 14회 금융문자 분석 경진대회

### WonjunPark / import.위드

### 2019년 12월 28일


In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/dacon

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [0]:
cd /content/gdrive/My Drive/dacon

/content/gdrive/My Drive/dacon


In [0]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
train.head(2)

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [0]:
test = pd.read_csv("public_test.csv")

test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [0]:
submission=pd.read_csv("submission_제출양식.csv")

submission.head(2)

,id,smishing
0,340000,1.000000e-09
1,340001,1.000000e-09


In [0]:
train.shape, test.shape, submission.shape

((295945, 4), (1626, 3), (1626, 2))

**Under sampling**

In [0]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [0]:
19646/train.shape[0]

0.0663839564783997

In [0]:
850/12600

0.06746031746031746

In [0]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [0]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [0]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [0]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((12600, 4), (12600, 1), (1626, 1), (1626, 1))

**Mecab 다운로드 및 사용**

In [0]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 16), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [0]:
ls

Mecab-ko-for-Google-Colab/  public_test.csv  submission_제출양식.csv  train.csv


In [0]:
cd Mecab-ko-for-Google-Colab/

/content/gdrive/My Drive/dacon/Mecab-ko-for-Google-Colab


In [0]:
ls

images/  install_mecab-ko_on_colab190912.sh  README.md


In [0]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 491kB 41.4MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
  Created wheel for JPype1: filename=JPype1-0.7.1-cp36-cp36m-linux_x86_64.whl size=2449046 sha256=490586abf679fb63fed9541807db237d4078415f8a49b02f32068d5d1ed44555
  Stored in directory: /root/.cache/pip/wheels/b0/49/6a/4393ef8542c00becf80691bd242693db9e263d6e499323a984
Successfully built JPype1
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2019-12-27 16:27:18--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket

In [0]:
import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

In [0]:
train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

12600it [00:06, 1960.56it/s]
1626it [00:01, 1321.65it/s]


In [0]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [0]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [0]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [0]:
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
vec_x_test= v.transform(X_test).toarray()

In [0]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_train)
y_train_pred1_one= [ i[1]  for i in y_train_pred1]

y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

In [0]:
!ls "/content/gdrive/My Drive/dacon" 

Mecab-ko-for-Google-Colab  public_test.csv  submission_제출양식.csv  train.csv


In [0]:
cd /content/gdrive/My Drive/dacon

/content/gdrive/My Drive/dacon


In [0]:
submission['smishing'] = y_test_pred1_one

submission.to_csv("14th_baseline_multi.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장